## **Pomocne moduly**

In [150]:
import itertools
import math
import numpy as np
import UQpy
from scipy.special import comb
from UQpy.distributions import Uniform, JointIndependent
from UQpy.surrogates.polynomial_chaos import PolynomialChaosExpansion, TotalDegreeBasis, LeastSquareRegression
from UQpy.sensitivity import PceSensitivity
from sklearn.metrics import mean_squared_error, r2_score
import time  

Generování multiindexových množin

In [153]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb  6 12:57:36 2023

@author: D. Loukrezis
"""

import itertools
import math
import numpy as np
from scipy.special import comb


def setsize(N, w):
    """Vypočítá počet PCE polynomů pro N dimenzí a maximální stupeň w pomocí kombinatoriky"""
    return int(comb(N+w-1, N-1))    


def td_set_recursive(N, w, rows):
    """Rekurzivní funkce pro generování multiindexů totálního stupně"""
    if N == 1:  # Základní případ - 1D
        subset = w*np.ones([rows, 1])
    else:
        if w == 0:  # Nulový stupeň - všechny nuly
            subset = np.zeros([rows, N])
        elif w == 1:  # Stupeň 1 - jednotková matice
            subset = np.eye(N)
        else:
            # Inicializace podmatice
            subset = np.empty([rows, N])
            
            # Počáteční řádek podmatice
            row_start = 0
            
            # Iterace přes polynomiální stupně
            for k in range(0, w+1):
                
                # Počet řádků podmatice
                sub_rows = setsize(N-1, w-k)
                
                # Aktualizace koncového řádku
                row_end = row_start + sub_rows - 1
                
                # První sloupec - nastaví hodnotu k
                subset[row_start:row_end+1, 0] = k*np.ones(sub_rows)
                
                # Rekurzivní volání pro zbylé dimenze
                subset[row_start:row_end+1, 1:] = td_set_recursive(N-1, w-k, 
                                                              sub_rows)
                                                                     
                # Aktualizace indexu řádku
                row_start = row_end + 1
    
    return subset


def td_multiindex_set(N, w):
    """Generuje multiindex set totálního stupně pro N parametrů a max stupeň w"""
    
    # Velikost množiny multiindexů
    td_size = int(comb(N+w, N))
    
    # Inicializace výsledné množiny
    midx_set = np.empty([td_size, N])
    
    # Počáteční řádek
    row_start = 0
    
    # Iterace přes polynomiální stupně
    for i in range(0, w+1):
        
        # Výpočet počtu řádků pro daný stupeň
        rows = setsize(N, i)
        
        # Aktualizace koncového řádku
        row_end = rows + row_start - 1
        
        # Rekurzivní generování části množiny
        midx_set[row_start:row_end+1, :] = td_set_recursive(N, i, rows)
        
        # Aktualizace počátečního řádku
        row_start = row_end + 1
        
    return midx_set.astype(int)  # Převod na celá čísla


def tp_multiindex_set(N, w):
    """Generuje tenzorový součin multiindex set pro N parametrů a max stupeň w"""
    orders = np.arange(0, w+1, 1).tolist()  # Seznam stupňů 0 až w
    if N == 1:  # Speciální případ pro 1D
        midx_set = np.array(list(map(lambda el:[el], orders)))
    else:
        # Generování všech kombinací (kartézský součin)
        midx = list(itertools.product(orders, repeat=N))
        midx = [list(elem) for elem in midx]
        # Výpočet součtů pro každý multiindex
        midx_sums = [int(math.fsum(midx[i])) for i in range(len(midx))]
        # Seřazení podle součtu (totální stupeň)
        midx_sorted = sorted(range(len(midx_sums)), 
                             key=lambda k: midx_sums[k])
        # Sestavení výsledného pole
        midx_set = np.array([midx[midx_sorted[i]] for i in range(len(midx))])   
    return midx_set.astype(int)  # Převod na celá čísla

In [155]:
import numpy as np


def admissible_neighbors(index, index_set):
    """Vrátí přípustné sousední indexy pro daný index v monotonní množině indexů"""
    for_neighbors = forward_neighbors(index)  # Získá všechny "dopředné" sousedy
    # Filtruje jen přípustné sousedy (kde všechny zpětné sousedy jsou v index_set)
    for_truefalse = [is_admissible(fn, index_set) for fn in for_neighbors]
    adm_neighbors = np.array(for_neighbors)[for_truefalse].tolist()  # Konverze na seznam
    return adm_neighbors


def is_admissible(index, index_set):
    """Kontroluje, zda je index přípustný (všechny jeho zpětné sousedy obsahuje index_set)"""
    back_neighbors = backward_neighbors(index)  # Získá všechny "zpětné" sousedy
    for ind_b in back_neighbors:  # Pro každého zpětného souseda
        if ind_b not in index_set:  # Pokud není v množině, index není přípustný
            return False
    return True  # Všechny zpětné sousedy jsou v množině


def forward_neighbors(index):
    """Vrátí dopředné sousedy - každou dimenzi zvýší o 1 (např. (2,1) → (3,1), (2,2))"""
    N = len(index)  # Počet dimenzí multiindexu
    for_neighbors = []
    for i in range(N):  # Pro každou dimenzi
        index_tmp = index[:]  # Kopie původního indexu
        index_tmp[i] = index_tmp[i] + 1  # Inkrementace aktuální dimenze
        for_neighbors.append(index_tmp)  # Přidání do seznamu sousedů
    return for_neighbors


def backward_neighbors(index):
    """Vrátí zpětné sousedy - každou dimenzi sníží o 1 (pokud možno, např. (2,2) → (1,2), (2,1))"""
    N = len(index)  # Počet dimenzí multiindexu
    back_neighbors = []
    for i in range(N):  # Pro každou dimenzi
        index_tmp = index[:]  # Kopie původního indexu
        if index_tmp[i] > 0:  # Pokud lze snížit (nenulová hodnota)
            index_tmp[i] = index_tmp[i] - 1  # Dekrementace aktuální dimenze
            back_neighbors.append(index_tmp)  # Přidání do seznamu sousedů
    return back_neighbors

In [157]:
!pip install UQpy
!pip install --upgrade UQpy
!pip install torch

## **PCE** = OPENTURNS

In [159]:
# -*- coding: utf-8 -*-
import openturns as ot
import numpy as np
import scipy as sp

class PolynomialChaosExpansion():
    
    def __init__(self, pdf, exp_design_in, exp_design_out):
        
        # case without given pdf: identify PDF from data
        if pdf == None:
            chaos_algo_data = ot.FunctionalChaosAlgorithm(exp_design_in, 
                                                          exp_design_out)
            chaos_algo_data.run()
            self.pdf = chaos_algo_data.getDistribution()
        else:    
            self.pdf = pdf
        self.num_inputs = self.pdf.getDimension()
        self.num_outputs = exp_design_out.shape[1]
        self.num_samples = exp_design_out.shape[0]
        self.exp_design_inputs = exp_design_in
        self.exp_design_outputs = exp_design_out
        
        # get enumerate function for single and multi-indices
        self.enumerate_function = ot.LinearEnumerateFunction(self.num_inputs)
        
        # assign the correct polynomials based on the input distribution
        self.polynomial_collection = [
                  ot.StandardDistributionPolynomialFactory(
                      self.pdf.getMarginal(i))
                  for i in range(self.num_inputs)]
        
        # get a general product basis, to be used for the construction of 
        # specific PCE bases, i.e., given specific index sets
        self.product_basis = ot.OrthogonalProductPolynomialFactory(
                        self.polynomial_collection, self.enumerate_function)
        
        # get transformation function - very important!
        self.transformation = ot.DistributionTransformation(
                                self.pdf, self.product_basis.getMeasure())
    
    def set_multi_index_set(self, multi_index_set):
        self.multi_index_set = multi_index_set
        self.single_index_set = [self.enumerate_function.inverse(idx) 
                                  for idx in self.multi_index_set]
        
    def set_single_index_set(self, single_index_set):
        self.single_index_set = single_index_set
        self.multi_index_set = [list(self.enumerate_function(idx)) 
                                     for idx in self.single_index_set]
        
    def construct_basis(self):
        self.basis = self.product_basis.getSubBasis(self.single_index_set)
        self.num_polynomials = self.basis.getSize()
        
    def set_exp_design(self, exp_design_in, exp_design_out):
        if self.num_outputs != exp_design_out.shape[1]:
            raise ValueError('Output dimensions do not agree!')
        self.exp_design_inputs = exp_design_in
        self.exp_design_outputs = exp_design_out 
        self.num_samples = exp_design_out.shape[0]
        
    def evaluate_basis(self, design_in):
        n_samples, n_inputs = np.shape(design_in)
        if n_inputs != self.num_inputs:
            raise ValueError('Input dimensions do not agree!')
        # transform input data
        design_in_tf = np.array(self.transformation(design_in))
        # compute basis evaluation matrix
        eval_matrix = np.array([self.basis[j](design_in_tf) 
                                for j in range(self.num_polynomials)])
        # remove superficial dimension
        eval_matrix = np.squeeze(eval_matrix)
        return eval_matrix.T
    
    def compute_design_matrix(self):
        self.design_matrix = self.evaluate_basis(self.exp_design_inputs)
    
    def compute_coefficients(self):
        # compute design matrix
        self.compute_design_matrix()
        # compute PCE coefficients using least squares regression
        self.coefficients, _, _, singular_values = np.linalg.lstsq(
                                                      self.design_matrix, 
                                                      self.exp_design_outputs,
                                                      rcond=None)
        self.condition_number = singular_values[0] / singular_values[-1]
    
    def predict(self, design_in):
        eval_matrix = self.evaluate_basis(design_in)
        return eval_matrix.dot(self.coefficients)
    
    def compute_mean(self):
        return self.coefficients[0]
    
    def compute_variance(self):
        return np.sum(np.square(self.coefficients[1:]), axis=0)
    
    def compute_sobol_first(self):
     
        sobol_f = np.empty([self.num_inputs, self.num_outputs])
        variance = self.compute_variance()
        # remove zeroth multi-index from multi-index set
        midx_minus_0 = np.delete(self.multi_index_set, 0, axis=0)
        for i in range(self.num_inputs):
            # remove i-th column from multi-index set without 0
            midx_minus_i = np.delete(midx_minus_0, i, axis=1)
            # get the rows with all indices equal to zero
            row_sum = np.sum(midx_minus_i, axis=1)
            zero_rows = np.asarray(np.where(row_sum==0)).flatten() + 1
            partial_variance = np.sum(np.square(self.coefficients[zero_rows]),
                                      axis=0)
            sobol_f[i,:] = partial_variance / variance
        return sobol_f
    
    def compute_sobol_total(self):

        sobol_t = np.empty([self.num_inputs, self.num_outputs])
        variance = self.compute_variance()
        mis = np.array(self.multi_index_set)
        for i in range(self.num_inputs):
            # we want all multi-indices where the i-th index is NOT zero  
            idx_column_i = mis[:,i] 
            non_zero_rows = np.asarray(np.where(idx_column_i!=0)).flatten()
            partial_variance = np.sum(
                                np.square(self.coefficients[non_zero_rows]),
                                axis=0)
            sobol_t[i,:] = partial_variance / variance
        return sobol_t
    
    def compute_generalized_sobol_first(self):
        # compute variance and first order Sobol indices (elementwise)
        variance = self.compute_variance()
        sobol_f = self.compute_sobol_first()
        # retrieve elementwise partial variances
        partial_variances = sobol_f*variance
        # compute aggregated variance (scalar value)
        aggregated_variance = np.sum(variance)
        # compute aggregated partial variance per input 
        # 1d array with length equal to num_inputs
        aggregated_partial_variances = np.sum(partial_variances, axis=1)
        # compute generalized first order Sobol indices
        sobol_f_gen = aggregated_partial_variances / aggregated_variance
        return sobol_f_gen

    def compute_generalized_sobol_total(self):
        # compute variance and total order Sobol indices (elementwise)
        variance = self.compute_variance()
        sobol_t = self.compute_sobol_total()
        # retrieve elementwise partial variances
        partial_variances = sobol_t*variance
        # compute aggregated variance (scalar value)
        aggregated_variance = np.sum(variance)
        # compute aggregated partial variance per input 
        # 1d array with length equal to num_inputs
        aggregated_partial_variances = np.sum(partial_variances, axis=1)
        # compute generalized total order Sobol indices
        sobol_t_gen = aggregated_partial_variances / aggregated_variance
        return sobol_t_gen




In [160]:
# Testuje samotné PCE
"""
num_samples = 500
marginals = [Uniform(0, 5), Uniform(0, 2)] 
input_distribution = JointIndependent(marginals)
input_sample = input_distribution.rvs(num_samples)
print("Generovaná vstupní data:")
print(input_sample)

def test_function(X):
    x1, x2 = X[:, 0], X[:, 1]
    return np.expand_dims(2*x1 + 3*x2**2, axis=1)

output_sample = test_function(input_sample)

pce_uqpy = PolynomialChaosExpansionUQPY(input_distribution, input_sample, output_sample)

test_inputs = input_distribution.rvs(10)
predictions = pce_uqpy.predict(test_inputs)

mean_value = pce_uqpy.compute_mean()
variance_value = pce_uqpy.compute_variance()

sobol_first = pce_uqpy.compute_sobol_first()
sobol_total = pce_uqpy.compute_sobol_total()

print("První řád Sobolových indexů:")
print(sobol_first)
print("Celkové Sobolovy indexy:")
print(sobol_total)
print("Střední hodnota modelu:")
print(mean_value)
print("Rozptyl modelu:")
print(variance_value)

true_outputs = test_function(test_inputs)
print("Skutečné hodnoty vs Predikované hodnoty:")
for i in range(len(test_inputs)):
    print(f"Vstupy: {test_inputs[i]}, Skutečná hodnota: {true_outputs[i][0]}, Predikovaná hodnota: {predictions[i][0]}")
    
"""

'\nnum_samples = 500\nmarginals = [Uniform(0, 5), Uniform(0, 2)] \ninput_distribution = JointIndependent(marginals)\ninput_sample = input_distribution.rvs(num_samples)\nprint("Generovaná vstupní data:")\nprint(input_sample)\n\ndef test_function(X):\n    x1, x2 = X[:, 0], X[:, 1]\n    return np.expand_dims(2*x1 + 3*x2**2, axis=1)\n\noutput_sample = test_function(input_sample)\n\npce_uqpy = PolynomialChaosExpansionUQPY(input_distribution, input_sample, output_sample)\n\ntest_inputs = input_distribution.rvs(10)\npredictions = pce_uqpy.predict(test_inputs)\n\nmean_value = pce_uqpy.compute_mean()\nvariance_value = pce_uqpy.compute_variance()\n\nsobol_first = pce_uqpy.compute_sobol_first()\nsobol_total = pce_uqpy.compute_sobol_total()\n\nprint("První řád Sobolových indexů:")\nprint(sobol_first)\nprint("Celkové Sobolovy indexy:")\nprint(sobol_total)\nprint("Střední hodnota modelu:")\nprint(mean_value)\nprint("Rozptyl modelu:")\nprint(variance_value)\n\ntrue_outputs = test_function(test_inputs

## **SAPCE**

In [162]:
class SensitivityAdaptivePCE:
    def __init__(self, pdf, exp_design_in, exp_design_out, max_partial_degree=10, tolerance=1e-3):
        self.pdf = pdf if pdf is not None else JointIndependent([Uniform(0, 1) for _ in range(exp_design_in.shape[1])])
        self.exp_design_in = exp_design_in
        self.exp_design_out = exp_design_out
        self.max_partial_degree = max_partial_degree
        self.tolerance = tolerance   # Ukládání do self zajišťuje, že hodnoty přežijí mimo __init__ a jsou dostupné v celé třídě.

    
        num_inputs = self.pdf.getDimension() #Zjistí počet vstupních proměnných z pravděpodobnostního rozdělení 
        td1_set = td_multiindex_set(num_inputs, 1).tolist() #Vytvoří množinu multiindexů pro polynomy 1. stupně
        self.pce = PolynomialChaosExpansion(self.pdf, self.exp_design_in, self.exp_design_out) #Inicializuje PCE model, který bude aproximovat vztah mezi vstupy a výstupy
        self.pce.set_multi_index_set(td1_set) #Nastaví množinu multiindexů definovanou v  td1_set
        self.pce.construct_basis() #Sestaví ortogonální polynomiální bázi na základě
        self.pce.compute_coefficients() #Spočítá koeficienty PCE metodou nejmenších čtverců (LSQ) z dat.
         
        self.active_multi_indices = [self.pce.multi_index_set[0]]  #Začíná s nulovým multi-indexem
        self.admissible_multi_indices = self.pce.multi_index_set[1:] #Inicializace přípustných multi-indexůObsahuje všechny multi-indexy 1. stupně
        admissible_coefficients = self.pce.coefficients[1:].tolist() # Pro každý přípustný multi-index spočítá součet absolutních hodnot jeho koeficientů
        aggregated_admissible_coefficients = np.sum(np.abs(admissible_coefficients), axis=1) #absolutni hodnota, bez ohledu na zanemenko
       
        help_index = np.argmax(aggregated_admissible_coefficients) #Najde index (pozici) polynomu s největším součtem absolutních hodnot koeficientů
        max_admissible_multi_index = self.admissible_multi_indices.pop(help_index) #Odstrani a ulozi max. Algoritmus pak nemusí procházet již vybrané členy
        self.active_multi_indices.append(max_admissible_multi_index)  #Aktualizace aktivních multi-indexů
        print("Jsem tu v sapce")
           
    def construct_adaptive_basis(self, max_condition_number=1e2, termination_info=True):
        while True:
            # T1: Kontroluje, zda číslo podmíněnosti aktuální matice překročilo limit
            if self.pce.condition_number > max_condition_number:
                if termination_info:
                    print("Adaptive basis construction terminated:" 
                        + " design matrix not sufficiently well-conditioned.")
                break
            
            # Hledání nových přípustných multi-indexů
            new_admissible_multi_indices = admissible_neighbors(
                                            self.active_multi_indices[-1],
                                            self.active_multi_indices)
            
            # T2: Zajišťuje, že všechny nové přípustné multi-indexy splňují podmínku maximálního parciálního stupně, Stupně polynomu
    
            for idx, adm_multi_Indcs in reversed(list(enumerate(new_admissible_multi_indices))):
                for adm_multi_Indx in adm_multi_Indcs:
                    if adm_multi_Indx > self.max_partial_degree:
                        new_admissible_multi_indices.pop(idx)
                            
            #Kontroluje, zda číslo podmíněnosti aktuální matice překročilo limit
            if [self.max_partial_degree]*self.pce.num_inputs in self.active_multi_indices: 
                if len(new_admissible_multi_indices) == 0:
                    if termination_info:
                        print("Adaptive basis construction terminated:" 
                        + " maximum partial degree reached.")
                    break
            
            # Kontroluje, zda počet členů báze nepřekročil počet trénovacích vzorků
            num_terms = len(self.active_multi_indices) + \
                        len(self.admissible_multi_indices) +\
                        len(new_admissible_multi_indices)
            if num_terms >= len(self.pce.exp_design_inputs):
                if termination_info:
                    print("Adaptive basis construction terminated:" 
                        + " basis cardinality reached experimental design size.")
                break
             
            self.admissible_multi_indices += new_admissible_multi_indices  #Přidá nově nalezené přípustné multi-indexy do stávající množiny přípustných indexů
            all_multi_indices = self.active_multi_indices + self.admissible_multi_indices
                                                                    
            self.pce.set_multi_index_set(all_multi_indices) #Nastaví kompletní množinu indexů do PCE modelu
            self.pce.construct_basis()
            self.pce.compute_coefficients()
            
            # Výběr a přesun nejvýznamnějšího indexu
            idx = len(self.active_multi_indices)  
            admissible_coefficients = self.pce.coefficients[idx:].tolist()
            aggregated_admissible_coefficients = np.sum(np.abs(admissible_coefficients), axis=1)
            help_index = np.argmax(aggregated_admissible_coefficients)
            max_admissible_multi_index = self.admissible_multi_indices.pop(help_index)
            self.active_multi_indices.append(max_admissible_multi_index)

    def construct_active_pce(self):
        pce = PolynomialChaosExpansion(self.pdf, 
                            self.exp_design_in, 
                            self.exp_design_out)
        pce.set_multi_index_set(self.active_multi_indices)
        pce.construct_basis()
        pce.compute_coefficients()
        return pce
    
    # nový polynomiální chaosový expanzní (PCE) model, který kombinuje aktuální aktivní i přípustné multi-indexy
    # Není to jen "srovnání" – přímo ovlivňuje sestavu týmu!
    def construct_augmented_pce(self):
        pce = PolynomialChaosExpansion(self.pdf, 
                                    self.exp_design_in, 
                                    self.exp_design_out)
        pce.set_multi_index_set(self.active_multi_indices + 
                            self.admissible_multi_indices)
        pce.construct_basis()
        pce.compute_coefficients()
        return pce      

    # Vytvoří optimalizovanou verzi rozšířeného PCE modelu s kontrolou stability.
    def construct_reduced_augmented_pce(self, max_condition_number=1e2):
        # compute augmented pce
        pce = self.construct_augmented_pce()
        while True:
            # exit if the condition number is acceptable
            if pce.condition_number <= max_condition_number and\
                len(pce.multi_index_set) <= len(pce.exp_design_inputs):
                break
            # remove single- and multi-index with minimum contribution
            idx_min = np.argmin(np.sum(np.abs(pce.coefficients), axis=1))
            pce.multi_index_set.pop(idx_min)
            pce.single_index_set.pop(idx_min)
            # compute pce basis and coefficients with reduced multi-index set
            pce.construct_basis()
            pce.compute_coefficients()
        # re-order multi-indices and coefficients
        # coeffs = pce.coefficients
        # midx = np.array(pce.multi_index_set)
        # coeffs_aggr = np.sum(np.abs(coeffs), axis=1)
        # order_idx = np.flip(np.argsort(coeffs_aggr))
        # midx_ord = midx[order_idx, :].tolist()
        # pce.set_multi_index_set(midx_ord)
        # pce.construct_basis()
        # pce.compute_coefficients()
        return pce

In [163]:
import openturns as ot
import numpy as np
import time
from sklearn.metrics import mean_squared_error

def root_mean_squared_error(y_true, y_pred, multioutput='uniform_average'):
    mse = mean_squared_error(y_true, y_pred, multioutput=multioutput)
    if isinstance(mse, np.ndarray):
        return np.sqrt(mse)
    return np.sqrt(mse)

def test_SensitivityAdaptivePCE():
    # Nastavení parametrů
    dim = 3  # Počet vstupních proměnných
    size = 1000  # Počet trénovacích vzorků
    test_size = 50  # Počet testovacích vzorků
    
    # Vytvoření rovnoměrného rozdělení
    marginals = [ot.Uniform(0, 1) for _ in range(dim)]
    distribution = ot.ComposedDistribution(marginals)
    
    # Generování trénovacích dat
    experiment = ot.MonteCarloExperiment(distribution, size)
    exp_design_in = experiment.generate()
    
    # Testovací model s více výstupy
    def model(x):
        x_np = np.array(x)
        return np.column_stack((
            x_np[:, 0] + 2 * x_np[:, 1] + 3 * x_np[:, 2],
            x_np[:, 0] * x_np[:, 1] + x_np[:, 2],
            np.sin(x_np[:, 0]) + np.cos(x_np[:, 1]),
            np.exp(x_np[:, 2])
        ))
    
    exp_design_out = model(exp_design_in)
    
    # Generování testovacích dat
    test_experiment = ot.MonteCarloExperiment(distribution, test_size)
    test_in = test_experiment.generate()
    test_out = model(test_in)
    
    # Vytvoření a trénink adaptivního PCE modelu
    t0 = time.time()
    sapce = SensitivityAdaptivePCE(
        pdf=distribution, 
        exp_design_in=exp_design_in, 
        exp_design_out=exp_design_out, 
        max_partial_degree=10,
        tolerance=1e-3
    )
    sapce.construct_adaptive_basis(max_condition_number=1e3)
    pce = sapce.pce  # Předpokládáme, že třída SensitivityAdaptivePCE má tento atribut
    
    # Předpovědi na testovacích datech
    pce_predictions = pce.predict(test_in)
    
    # Vyhodnocení přesnosti
    vector_rmse = root_mean_squared_error(test_out, pce_predictions, multioutput='raw_values')
    avg_rmse = root_mean_squared_error(test_out, pce_predictions, multioutput='uniform_average')
    
    # Výpočet R² skóre
    r2 = []
    for j in range(test_out.shape[1]):
        ss_res = np.sum((test_out[:, j] - pce_predictions[:, j])**2)
        ss_tot = np.sum((test_out[:, j] - np.mean(test_out[:, j]))**2)
        r2.append(1 - ss_res/ss_tot)
    
    # Výstupy modelu
    pce_mean = pce.compute_mean()
    pce_variance = pce.compute_variance()
    pce_std = np.sqrt(pce_variance)
    
    # Výpis výsledků
    print("\n--- Přesnost modelu ---")
    print(f"Průměrná RMSE: {avg_rmse:.4f}")
    print("\nRMSE pro jednotlivé výstupy:")
    for i, rmse in enumerate(vector_rmse):
        print(f"  Výstup {i+1}: {rmse:.4f}")
    
    print("\nR² skóre pro jednotlivé výstupy:")
    for j, score in enumerate(r2):
        print(f"  Výstup {j+1}: {score:.4f}")
    s
    print("\n--- Statistické charakteristiky ---")
    print("Střední hodnoty výstupů:")
    print(pce_mean)
    print("\nRozptyly výstupů:")
    print(pce_variance)
    print("\nSměrodatné odchylky výstupů:")
    print(pce_std)
    
    print("\n--- Informace o modelu ---")
    print(f"Počet vstupů: {dim}")
    print(f"Trénovacích vzorků: {size}")
    print(f"Testovacích vzorků: {test_size}")
    print(f"Čas výpočtu: {time.time()-t0:.2f} s")

# Spuštění testu
test_SensitivityAdaptivePCE()

Jsem tu v sapce
Adaptive basis construction terminated: design matrix not sufficiently well-conditioned.

--- Přesnost modelu ---
Průměrná RMSE: 0.0000

RMSE pro jednotlivé výstupy:
  Výstup 1: 0.0000
  Výstup 2: 0.0000
  Výstup 3: 0.0000
  Výstup 4: 0.0000

R² skóre pro jednotlivé výstupy:
  Výstup 1: 1.0000
  Výstup 2: 1.0000
  Výstup 3: 1.0000
  Výstup 4: 1.0000

--- Statistické charakteristiky ---
Střední hodnoty výstupů:
[3.         0.75       1.30116868 1.71828183]

Rozptyly výstupů:
[1.16666667 0.13194444 0.08060461 0.24203561]

Směrodatné odchylky výstupů:
[1.08012345 0.36324158 0.28390951 0.49197114]

--- Informace o modelu ---
Počet vstupů: 3
Trénovacích vzorků: 1000
Testovacích vzorků: 50
Čas výpočtu: 122.65 s
